In [1]:
import numpy as np

import pandas as pd







import os

for dirname, _, filenames in os.walk('/home/featurize/data'):

    for filename in filenames:

        print(os.path.join(dirname, filename))

        

#确认文件路径

In [2]:
import os

import warnings

import numpy as np 

import pandas as pd 

from sklearn.linear_model import Lasso

from sklearn.linear_model import LassoCV

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, LabelEncoder



print(os.listdir("/kaggle/input"))

%matplotlib inline



#遍历文件名称

['house-prices-advanced-regression-techniques']


In [3]:
from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

from sklearn.ensemble import RandomForestRegressor

from catboost import  CatBoostRegressor






In [4]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv') 
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv') 
sample = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [5]:
#Function for viewing data with max 1000 columns and rows

def display_all(df):

    with pd.option_context("display.max_rows", 10, "display.max_columns", 10): 

        display(df)

In [6]:
#Seprate taget value "Id" "Saleprice(more importantly)" from the features

y     = train[['Id','SalePrice']]

#After storing the target value "SalePrice" in y we can now drop it from the feature set 

train = train.drop('SalePrice',axis=1)

In [7]:
#creates a list with both Training and Testing Data sets -> all_dfs

all_dfs = [train,test]

# "d.concat(all_dfs)" merges the train and test datasets into a single DataFrame called all_df. It stacks them vertically by default since they share the same columns.

# ".reset_index(drop=True)" resets the index of the new concatenated DataFrame and drops the old index. 

# Without this, the concatenated DataFrame would retain the indices from both train and test

all_df = pd.concat(all_dfs).reset_index(drop=True);

In [8]:
display_all((all_df.isnull().sum()/all_df.shape[0])*100)

Id                0.000000
MSSubClass        0.000000
MSZoning          0.137033
LotFrontage      16.649538
LotArea           0.000000
                   ...    
MiscVal           0.000000
MoSold            0.000000
YrSold            0.000000
SaleType          0.034258
SaleCondition     0.000000
Length: 80, dtype: float64

In [9]:
# Drop all the features with very high number in null rows 

all_df.drop(['Alley','PoolQC','MiscFeature','Fence','FireplaceQu','Utilities'],axis=1,inplace=True)

In [10]:
# with all feature with few null views, we manually filled the missing data

all_df['LotFrontage'].fillna(value=all_df['LotFrontage'].median(),inplace=True)

all_df['MasVnrType'].fillna(value='None',inplace=True)

all_df['MasVnrArea'].fillna(0,inplace=True)

all_df['BsmtCond'].fillna(value='TA',inplace=True)

all_df['BsmtExposure'].fillna(value='No',inplace=True)

all_df['Electrical'].fillna(value='SBrkr',inplace=True)

all_df['BsmtFinType2'].fillna(value='Unf',inplace=True)

all_df['GarageType'].fillna(value='Attchd',inplace=True)

all_df['GarageYrBlt'].fillna(value=all_df['GarageYrBlt'].median(),inplace=True)

all_df['GarageFinish'].fillna(value='Unf',inplace=True)

all_df['GarageQual'].fillna(value='TA',inplace=True)

all_df['GarageCond'].fillna(value='TA',inplace=True)

all_df['BsmtFinType1'].fillna(value='NO',inplace=True)

all_df['BsmtQual'].fillna(value='No',inplace=True)

all_df['BsmtFullBath'].fillna(value=all_df['BsmtFullBath'].median(),inplace=True)

all_df['BsmtFinSF1'].fillna(value=all_df['BsmtFinSF1'].median(),inplace=True)

all_df['BsmtFinSF2'].fillna(value=0,inplace=True)

all_df['BsmtUnfSF'].fillna(value=0,inplace=True)

all_df['TotalBsmtSF'].fillna(value=all_df['TotalBsmtSF'].median(),inplace=True)

all_df['BsmtHalfBath'].fillna(value=0,inplace=True)

all_df['GarageCars'].fillna(value=all_df['GarageCars'].median(),inplace=True)

all_df['GarageArea'].fillna(value=all_df['GarageArea'].median(),inplace=True)

/tmp/ipykernel_17/114349460.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df['LotFrontage'].fillna(value=all_df['LotFrontage'].median(),inplace=True)
/tmp/ipykernel_17/114349460.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(va

In [11]:
#Using labelencoder from Ssklearn to convert "categorical features" to to a numerical format

#This step is essential before feeding the data into Machine Learning models since they work better with numerical input

labelencoder=LabelEncoder()



all_df['MSZoning']      = labelencoder.fit_transform(all_df['MSZoning'].astype(str))

all_df['Exterior1st']   = labelencoder.fit_transform(all_df['Exterior1st'].astype(str))

all_df['Exterior2nd']   = labelencoder.fit_transform(all_df['Exterior2nd'].astype(str))

all_df['KitchenQual']   = labelencoder.fit_transform(all_df['KitchenQual'].astype(str))

all_df['Functional']    = labelencoder.fit_transform(all_df['Functional'].astype(str))

all_df['SaleType']      = labelencoder.fit_transform(all_df['SaleType'].astype(str))

all_df['Street']        = labelencoder.fit_transform(all_df['Street'])   

all_df['LotShape']      = labelencoder.fit_transform(all_df['LotShape'])   

all_df['LandContour']   = labelencoder.fit_transform(all_df['LandContour'])   

all_df['LotConfig']     = labelencoder.fit_transform(all_df['LotConfig'])   

all_df['LandSlope']     = labelencoder.fit_transform(all_df['LandSlope'])   

all_df['Neighborhood']  = labelencoder.fit_transform(all_df['Neighborhood'])   

all_df['Condition1']    = labelencoder.fit_transform(all_df['Condition1'])   

all_df['Condition2']    = labelencoder.fit_transform(all_df['Condition2'])   

all_df['BldgType']      = labelencoder.fit_transform(all_df['BldgType'])   

all_df['HouseStyle']    = labelencoder.fit_transform(all_df['HouseStyle'])   

all_df['RoofStyle']     = labelencoder.fit_transform(all_df['RoofStyle'])   

all_df['RoofMatl']      = labelencoder.fit_transform(all_df['RoofMatl'])    

all_df['MasVnrType']    = labelencoder.fit_transform(all_df['MasVnrType'])   

all_df['ExterQual']     = labelencoder.fit_transform(all_df['ExterQual'])  

all_df['ExterCond']     = labelencoder.fit_transform(all_df['ExterCond'])   

all_df['Foundation']    = labelencoder.fit_transform(all_df['Foundation'])   

all_df['BsmtQual']      = labelencoder.fit_transform(all_df['BsmtQual'])   

all_df['BsmtCond']      = labelencoder.fit_transform(all_df['BsmtCond'])   

all_df['BsmtExposure']  = labelencoder.fit_transform(all_df['BsmtExposure'])   

all_df['BsmtFinType1']  = labelencoder.fit_transform(all_df['BsmtFinType1'])   

all_df['BsmtFinType2']  = labelencoder.fit_transform(all_df['BsmtFinType2'])   

all_df['Heating']       = labelencoder.fit_transform(all_df['Heating'])   

all_df['HeatingQC']     = labelencoder.fit_transform(all_df['HeatingQC'])   

all_df['CentralAir']    = labelencoder.fit_transform(all_df['CentralAir'])   

all_df['Electrical']    = labelencoder.fit_transform(all_df['Electrical'])    

all_df['GarageType']    = labelencoder.fit_transform(all_df['GarageType'])  

all_df['GarageFinish']  = labelencoder.fit_transform(all_df['GarageFinish'])   

all_df['GarageQual']    = labelencoder.fit_transform(all_df['GarageQual'])  

all_df['GarageCond']    = labelencoder.fit_transform(all_df['GarageCond'])   

all_df['PavedDrive']    = labelencoder.fit_transform(all_df['PavedDrive'])  

all_df['SaleCondition'] = labelencoder.fit_transform(all_df['SaleCondition'])  

In [12]:
#scales features by removing the mean

Scaler = StandardScaler()

# ".fit_transform()" calculates the mean and standard deviation for scaling and then applies the transformation to the entire dataset (all_df)

all_scaled   = pd.DataFrame(Scaler.fit_transform(all_df))

#spliting th data into 50/50 :1460 -> firt 1460, 1460:2920 -> second 1460.

#as the training set consists of the first 1460 rows (train_scaled), while the test set is the next 1460 rows (test_scaled)

#train_scaled: This represents the feature set after feature scaling, which contains all the predictors (i.e., features)

train_scaled = pd.DataFrame(all_scaled[:1460])

test_scaled  = pd.DataFrame(all_scaled[1460:2920])

In [13]:
X = train_scaled

X_train, X_test, y_train, y_test = train_test_split(X, y['SalePrice'], test_size=0.1, random_state=42)

In [14]:


XGB1 = XGBRegressor(

    max_depth=3,       # maximum depth

    learning_rate=0.17851317118854057,  # 学习率

    n_estimators=319,  # boosting rounds

    reg_alpha=0.1070232678681972,  # alpha

    reg_lambda=9.21620230766857e-05,  # lambda

    min_child_weight=3,  # minimum child weight

    n_jobs=-1,  # 使用所有可用的 CPU

)                  #^^(A higher value makes the model more conservative by preventing the tree from growing too complex )

XGB1.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.17851317118854057,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=319, n_jobs=-1,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
XGB2 = XGBRegressor(

    max_depth=3,  # maximum depth

    learning_rate=0.0364507719136609,  # 学习率

    n_estimators=965,  # boosting rounds

    reg_alpha=0.00018131710013969177,  # alpha

    reg_lambda=2.3152099456723933e-07,  # lambda

    min_child_weight=3,  # minimum child weight

    n_jobs=-1  # 使用所有可用的 CPU

)                   #^^(A higher value makes the model more conservative by preventing the tree from growing too complex )

XGB2.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0364507719136609,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=965, n_jobs=-1,
             num_parallel_tree=None, random_state=None, ...)

In [16]:


RF1 = RandomForestRegressor(

    n_estimators=198,  # 决策树数量

    max_depth=18,  # 最大树深度

    min_samples_split=2,  # 最小样本分割数

    min_samples_leaf=2,  # 叶节点最小样本数

    max_features=0.7084875029768692,  # 最大特征比例

    bootstrap=False,  # 是否启用bootstrap

    random_state=42  # 保持结果可重复

)

RF1.fit(X_train, y_train)






RandomForestRegressor(bootstrap=False, max_depth=18,
                      max_features=0.7084875029768692, min_samples_leaf=2,
                      n_estimators=198, random_state=42)

In [17]:
RF2 = RandomForestRegressor(

    n_estimators=293,  # 决策树数量

    max_depth=20,  # 最大树深度

    min_samples_split=2,  # 最小样本分割数

    min_samples_leaf=1,  # 叶节点最小样本数

    max_features=0.496408431158935,  # 最大特征比例

    bootstrap=False,  # 是否启用bootstrap

    random_state=42  # 保持结果可重复

)



RF2.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=20,
                      max_features=0.496408431158935, n_estimators=293,
                      random_state=42)

In [18]:
xgb_tunning=False

In [19]:
import optuna

import numpy as np

from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split



# 假设 X 和 y 已经准备好

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



def objective(trial):

    params = {

        'max_depth': trial.suggest_int('max_depth', 2, 10),

        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),

        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),

        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),

        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),

        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),

        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),

        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0)

    }

    

    model = XGBRegressor(**params, n_jobs=-1)

    model.fit(X_train, y_train, 

              eval_set=[(X_val, y_val)], 

              eval_metric='rmse',  # 使用RMSE作为评估指标

              early_stopping_rounds=50, 

              verbose=False)

    

    # 对目标值和预测值取对数

    y_pred = model.predict(X_val)

    y_pred_log = np.log(y_pred + 1)  # 避免对数0的问题

    y_val_log = np.log(y_val + 1)



    # 计算对数 RMSE

    log_rmse = np.sqrt(mean_squared_error(y_val_log, y_pred_log))

    return log_rmse





if xgb_tunning==True:

    # 创建文件以保存试验结果

    output_file = "optuna_trials.txt"



    # 定义输出函数，将每个 trial 的信息保存到文件

    def save_trial_result(study, trial):

        with open(output_file, 'a') as f:

            f.write(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}. "

                    f"Best is trial {study.best_trial.number} with value: {study.best_value}.\n")



    # 使用Optuna进行超参数调优

    study = optuna.create_study(direction='minimize')

    study.optimize(objective, n_trials=100, callbacks=[save_trial_result])



    print("Best parameters:", study.best_params)

    print("Best Log RMSE:", study.best_value)



    # 使用最佳参数训练最终模型

    best_params = study.best_params

    final_model = XGBRegressor(**best_params, n_jobs=-1)

    final_model.fit(X_train, y_train)



    # 在验证集上评估最终模型

    y_pred = final_model.predict(X_val)

    y_pred_log = np.log(y_pred + 1)

    y_val_log = np.log(y_val + 1)

    final_log_rmse = np.sqrt(mean_squared_error(y_val_log, y_pred_log))

    print("Final model Log RMSE:", final_log_rmse)

In [20]:
rf_tuning = False

In [21]:


import optuna

import numpy as np

from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split



# 假设 X 和 y 已经准备好

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



def objective(trial):

    params = {

        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),  # 森林中的树木数目

        'max_depth': trial.suggest_int('max_depth', 2, 20),  # 树的最大深度

        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),  # 内部节点再分裂所需的最小样本数

        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),  # 叶子节点所需的最小样本数

        'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),  # 每次分裂时的最大特征数

        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])  # 是否有放回地采样

    }

    

    # 创建随机森林模型

    rf = RandomForestRegressor(**params, random_state=42, n_jobs=-1)

    

    # 训练模型

    rf.fit(X_train, y_train)

    

    # 在验证集上进行预测

    y_pred = rf.predict(X_val)

    

    # 对目标值和预测值取对数，避免对数为0的问题

    y_pred_log = np.log(y_pred + 1)

    y_val_log = np.log(y_val + 1)

    

    # 计算对数 RMSE

    log_rmse = np.sqrt(mean_squared_error(y_val_log, y_pred_log))

    return log_rmse



if rf_tuning == True:

    # 创建文件以保存试验结果

    output_file = "rf_optuna_trials.txt"



    # 定义输出函数，将每个 trial 的信息保存到文件

    def save_trial_result(study, trial):

        with open(output_file, 'a') as f:

            f.write(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}. "

                    f"Best is trial {study.best_trial.number} with value: {study.best_value}.\n")



    # 使用 Optuna 进行超参数调优

    study = optuna.create_study(direction='minimize')

    study.optimize(objective, n_trials=100, callbacks=[save_trial_result])



    # 输出最佳参数和结果

    print("Best parameters:", study.best_params)

    print("Best Log RMSE:", study.best_value)



    # 使用最佳参数训练最终模型

    best_params = study.best_params

    final_rf = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)

    final_rf.fit(X_train, y_train)



    # 在验证集上评估最终模型

    y_pred_final = final_rf.predict(X_val)

    y_pred_log = np.log(y_pred_final )

    y_val_log = np.log(y_val )

    final_log_rmse = np.sqrt(mean_squared_error(y_val_log, y_pred_log))

    print("Final model Log RMSE:", final_log_rmse)

In [22]:
#generate the predictions using ".predict" for the "test_scaled" for both models and store it to merge later

y_pred_xgb1  = pd.DataFrame( XGB1.predict(test_scaled))

y_pred_xgb2  = pd.DataFrame( XGB2.predict(test_scaled))

y_pred_rf1 = pd.DataFrame(RF1.predict(test_scaled))

y_pred_rf2 = pd.DataFrame(RF2.predict(test_scaled))

#create empty dataset set to store the result (with the same format as the sample_submission)

y_pred=pd.DataFrame()





y_pred['SalePrice'] = 0.42 * y_pred_xgb1[0]+0.42 * y_pred_xgb2[0] +0.08*y_pred_rf1[0]+0.08*y_pred_rf2[0]

#store the Id column in the dataset

y_pred['Id'] = test['Id']

In [23]:
#save dataset into a csv file for submission

y_pred.to_csv('submission.csv',index=False)